In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('read data through spark').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/15 16:38:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [13]:
import pandas as pd
from pyspark.sql.types import *
import pyspark.sql.types as T

dtypes = pd.read_csv('./schemas/paper.csv').to_records(index=False).tolist()
print(dtypes)
fields = [T.StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)


[('identifier', 'Integer'), ('title', 'String'), ('abstract', 'String'), ('year', 'Integer'), ('citations', 'Integer')]


In [ ]:
paper_test = spark.read.option('header', 'true').csv('./assets/parsedData/papers.csv', header=True, schema=schema)

In [ ]:
paper_test.show()

In [ ]:
dtypes = pd.read_csv('./schemas/affiliation.csv').to_records(index=False).tolist()
print(dtypes)
fields = [StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)

In [4]:
papers_pyspark = spark.read.option('header', 'true').csv('./assets/parsedData/papers.csv', inferSchema=True)

In [ ]:
papers_pyspark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in papers_pyspark.columns]
   ).show()

In [5]:
papers_pyspark.show()

+----------+--------------------+--------------------+--------------------+-----+--------------------+---------+--------+
|identifier|         paper_title|             authors|        affiliations| year|   publication_venue|citations|abstract|
+----------+--------------------+--------------------+--------------------+-----+--------------------+---------+--------+
|         1|Book Review: Disc...| Marjorie Richardson|                   -| 1998|       Linux Journal|        0|    null|
|         2|MOSFET table look...|                    |                    | 1984| Integration, the...|        0|    null|
|         3|The verification ...|    Virgil D. Gligor| Univ. of Marylan...| 1984| International Jo...|        0|    null|
|         4|Another view of f...| M. Gyssens;J. Pa...| Univ. of Antwerp...| 1984| International Jo...|        0|    null|
|         5|Entity-relationsh...| Sushil Jajodia;P...| University of Mi...| 1984| International Jo...|        0|    null|
|         6|The computer

In [6]:
aff_test = papers_pyspark.drop("paper_title", "authors", "year", "publication_venue", "citations", "abstract")

In [10]:
aff_test.show()

+--------+--------------------+
|paper_id|        affiliations|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3| Univ. of Marylan...|
|       4| Univ. of Antwerp...|
|       5| University of Mi...|
|       6|                   -|
|       7| Univ. of Kansas,...|
|       8| National Chiao T...|
|       9| University of De...|
|      10| Technical Univer...|
|      11| The Universtty o...|
|      12| Brown Umversity,...|
|      13| The Pennsylvania...|
|      14| Iona College, Ne...|
|      15| Univ. of Waterlo...|
|      16|  MIT, Cambridge, MA|
|      17| New York Univ., ...|
|      18| Mini Systems, In...|
|      19| Bell Laboratorie...|
|      20|                    |
+--------+--------------------+
only showing top 20 rows



In [8]:
aff_test=aff_test.withColumnRenamed("identifier","paper_id")

In [9]:
aff_test.count()

2092355

In [11]:
from pyspark.sql.functions import *
aff_test = aff_test.withColumn("affiliations", trim(aff_test.affiliations))

In [ ]:
aff_test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in aff_test.columns]
   ).show()

In [ ]:
aff_test.filter(aff_test.affiliations.contains('-')).collect()

In [14]:
emp_RDD = spark.sparkContext.emptyRDD()
 
# Create an expected schema
columns = StructType([StructField('paper_id',
                                  IntegerType(), True),
                    StructField('affiliation',
                                StringType(), True)])
 
# Create an empty RDD with expected schema
df_test = spark.createDataFrame(data = emp_RDD,
                           schema = columns)

In [17]:
df_test.show()

22/01/15 17:58:43 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB
22/01/15 17:58:44 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB
22/01/15 17:58:47 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB
22/01/15 17:59:04 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB


+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of Antwerp,...|
|       4|                   -|
|       5|University of Mis...|
|       5|                   -|
|       5|Ballistic Missile...|
|       6|                   -|
|       7|Univ. of Kansas, ...|
|       7|Louisiana State U...|
|       7|Univ. of Kansas, ...|
|       8|National Chiao Tu...|
|       8|National Chiao Tu...|
|       8|National Tsing Hu...|
|       9|University of Den...|
|      10|Technical Univers...|
|      11|The Universtty of...|
|      12|Brown Umversity, ...|
|      13|The Pennsylvania ...|
+--------+--------------------+
only showing top 20 rows



22/01/15 18:00:30 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB
22/01/15 18:00:32 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB
22/01/15 18:00:35 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB
22/01/15 18:00:49 WARN DAGScheduler: Broadcasting large task binary with size 1931.7 KiB


In [16]:
for rows in aff_test.select("paper_id", "affiliations").collect():
    if rows[1].split(";"):
        for element in rows[1].split(";"):
            print(rows[0], element)
            newRow = spark.createDataFrame([(rows[0], element)], columns)
            df_test = df_test.union(newRow)
            df_test.show()
    else:
        print(rows[0], rows[1])
        newRow = spark.createDataFrame([(rows[0], rows[1])], columns)
        df_test = df_test.union(newRow)
        df_test.show()

1 -


+--------+-----------+
|paper_id|affiliation|
+--------+-----------+
|       1|          -|
+--------+-----------+

2 
+--------+-----------+
|paper_id|affiliation|
+--------+-----------+
|       1|          -|
|       2|           |
+--------+-----------+

3 Univ. of Maryland, College Park
+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
+--------+--------------------+

4 Univ. of Antwerp, Wilrijk, Belgium
+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of Antwerp,...|
+--------+--------------------+

4 -
+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryla

+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of Antwerp,...|
|       4|                   -|
|       5|University of Mis...|
|       5|                   -|
|       5|Ballistic Missile...|
|       6|                   -|
|       7|Univ. of Kansas, ...|
|       7|Louisiana State U...|
|       7|Univ. of Kansas, ...|
|       8|National Chiao Tu...|
|       8|National Chiao Tu...|
|       8|National Tsing Hu...|
|       9|University of Den...|
|      10|Technical Univers...|
|      11|The Universtty of...|
|      12|Brown Umversity, ...|
+--------+--------------------+

13 The Pennsylvania State Univ., University Park, PA
+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of

+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of Antwerp,...|
|       4|                   -|
|       5|University of Mis...|
|       5|                   -|
|       5|Ballistic Missile...|
|       6|                   -|
|       7|Univ. of Kansas, ...|
|       7|Louisiana State U...|
|       7|Univ. of Kansas, ...|
|       8|National Chiao Tu...|
|       8|National Chiao Tu...|
|       8|National Tsing Hu...|
|       9|University of Den...|
|      10|Technical Univers...|
|      11|The Universtty of...|
|      12|Brown Umversity, ...|
|      13|The Pennsylvania ...|
+--------+--------------------+
only showing top 20 rows

14 Manhattan College, Bronx, NY


+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of Antwerp,...|
|       4|                   -|
|       5|University of Mis...|
|       5|                   -|
|       5|Ballistic Missile...|
|       6|                   -|
|       7|Univ. of Kansas, ...|
|       7|Louisiana State U...|
|       7|Univ. of Kansas, ...|
|       8|National Chiao Tu...|
|       8|National Chiao Tu...|
|       8|National Tsing Hu...|
|       9|University of Den...|
|      10|Technical Univers...|
|      11|The Universtty of...|
|      12|Brown Umversity, ...|
|      13|The Pennsylvania ...|
+--------+--------------------+
only showing top 20 rows

15 Univ. of Waterloo, Waterloo, Ont., Canada


...

22/01/15 17:54:45 WARN DAGScheduler: Broadcasting large task binary with size 1922.4 KiB
22/01/15 17:54:47 WARN DAGScheduler: Broadcasting large task binary with size 1922.4 KiB
22/01/15 17:54:50 WARN DAGScheduler: Broadcasting large task binary with size 1922.4 KiB
22/01/15 17:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1922.4 KiB


+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of Antwerp,...|
|       4|                   -|
|       5|University of Mis...|
|       5|                   -|
|       5|Ballistic Missile...|
|       6|                   -|
|       7|Univ. of Kansas, ...|
|       7|Louisiana State U...|
|       7|Univ. of Kansas, ...|
|       8|National Chiao Tu...|
|       8|National Chiao Tu...|
|       8|National Tsing Hu...|
|       9|University of Den...|
|      10|Technical Univers...|
|      11|The Universtty of...|
|      12|Brown Umversity, ...|
|      13|The Pennsylvania ...|
+--------+--------------------+
only showing top 20 rows

251 -


22/01/15 17:56:49 WARN DAGScheduler: Broadcasting large task binary with size 1927.0 KiB
22/01/15 17:56:52 WARN DAGScheduler: Broadcasting large task binary with size 1927.0 KiB
22/01/15 17:56:56 WARN DAGScheduler: Broadcasting large task binary with size 1927.0 KiB
22/01/15 17:57:11 WARN DAGScheduler: Broadcasting large task binary with size 1927.0 KiB


+--------+--------------------+
|paper_id|         affiliation|
+--------+--------------------+
|       1|                   -|
|       2|                    |
|       3|Univ. of Maryland...|
|       4|Univ. of Antwerp,...|
|       4|                   -|
|       5|University of Mis...|
|       5|                   -|
|       5|Ballistic Missile...|
|       6|                   -|
|       7|Univ. of Kansas, ...|
|       7|Louisiana State U...|
|       7|Univ. of Kansas, ...|
|       8|National Chiao Tu...|
|       8|National Chiao Tu...|
|       8|National Tsing Hu...|
|       9|University of Den...|
|      10|Technical Univers...|
|      11|The Universtty of...|
|      12|Brown Umversity, ...|
|      13|The Pennsylvania ...|
+--------+--------------------+
only showing top 20 rows

252 -


KeyboardInterrupt: 

In [ ]:
###from pyspark.sql.functions import desc, row_number, monotonically_increasing_id
###from pyspark.sql.window import Window
###aff_test = aff_test.withColumn('aff_id', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)